In [0]:
!pip3 install cleverhans

In [0]:
"""
This tutorial shows how to generate adversarial examples using FGSM
and train a model using adversarial training with Keras.
It is very similar to mnist_tutorial_tf.py, which does the same
thing but without a dependence on keras.
The original paper can be found at:
https://arxiv.org/abs/1412.6572
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os

import tensorflow as tf
from tensorflow.python.platform import flags
import numpy as np
import keras
from keras import backend

from cleverhans.attacks import FastGradientMethod
from cleverhans.dataset import MNIST
from cleverhans.loss import CrossEntropy
from cleverhans.train import train
from cleverhans.utils import AccuracyReport
#from cleverhans.utils_keras import cnn_model
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.utils_tf import model_eval

FLAGS = flags.FLAGS

NB_EPOCHS = 10
BATCH_SIZE = 256
LEARNING_RATE = .001
TRAIN_DIR = 'train_dir'
FILENAME = 'mnist.ckpt'
LOAD_MODEL = False

Using TensorFlow backend.


In [0]:
import warnings
with warnings.catch_warnings():
  warnings.simplefilter('ignore', category=ImportWarning)
  from sklearn import random_projection
  from sklearn.decomposition import PCA

  from keras.models import Sequential
  from keras.layers import Dense, Activation, Flatten, BatchNormalization, Lambda


  
def mlp_model(logits=False, input_ph=None, input_dim=28*28,
              channels=1, nb_units=100, nb_classes=10):
  """
  Defines a mlp model using Keras sequential model
  :param logits: If set to False, returns a Keras model, otherwise will also
                  return logits tensor
  :param input_ph: The TensorFlow tensor for the input
                  (needed if returning logits)
                  ("ph" stands for placeholder but it need not actually be a
                  placeholder)
  :param img_rows: number of row in the image
  :param img_cols: number of columns in the image
  :param channels: number of color channels (e.g., 1 for MNIST)
  :param nb_filters: number of convolutional filters per layer
  :param nb_classes: the number of output classes
  :return:
  """
  model = Sequential()

  if keras.backend.image_dim_ordering() == 'th':
    input_shape = (channels, input_dim)
  else:
    input_shape = (input_dim, channels)

  layers = [Dense(units=nb_units, input_dim=input_dim),
          BatchNormalization(),
          Dense(units=nb_units),
          Activation('relu'),
          BatchNormalization(),
          Dense(units=nb_units),
          Activation('relu'),
          BatchNormalization(),
          Dense(nb_classes)]
  
  for layer in layers:
    model.add(layer)

  if logits:
    logits_tensor = model(input_ph)
  model.add(Activation('softmax'))

  if logits:
    return model, logits_tensor, model.trainable_weights
  else:
    return model, model.trainable_weights
  
initial_weights = None
def rp(input_dim=28*28, projected_dim=400, channels=1, stop_grad='total', a='relu', name='rp_layer'):
  def init(shape, dtype=tf.float32):
    rp_transformer = random_projection.SparseRandomProjection(n_components=shape[1], dense_output=True, density=1/3)
    projection = rp_transformer._make_random_matrix(shape[0], shape[1]).todense()
    projection = tf.convert_to_tensor(projection, dtype=tf.float32)
    if stop_grad == 'full':
      projection = keras.backend.stop_gradient(projection)
    if stop_grad == 'partial':
      projection = tf.where(projection == 0, keras.backend.stop_gradient(projection), projection)
    global initial_weights
    initial_weights = projection
    return projection
  
  projection_layer = Dense(units=projected_dim, input_dim=input_dim, kernel_initializer=init, use_bias=False, activation=a, trainable=False, name=name)
  return projection_layer
  
def rp_mlp_model(logits=False, input_ph=None, input_dim=28*28,
              channels=1, nb_units=100, nb_classes=10):
  var_list = []
  model = Sequential()

  if keras.backend.image_dim_ordering() == 'th':
    input_shape = (channels, input_dim)
  else:
    input_shape = (input_dim, channels)

  #layers = [Dense(units=nb_units, input_dim=input_dim),
  layers=[rp(name='rp_layer1',projected_dim=80),
          Activation('relu'),
          BatchNormalization(),
          Dense(units=nb_units),
          Activation('relu'),
          BatchNormalization(),
          rp(name='rp_layer2',projected_dim=80),
          Activation('relu'),
          BatchNormalization(),
          Dense(units=nb_units),
          Activation('relu'),
          BatchNormalization(),
          Dense(nb_classes)]
  
  for layer in layers:
    model.add(layer)
  
  tvars = model.trainable_weights
  var_list = [var for var in tvars if 'rp_layer' not in var.name]

  if logits:
    logits_tensor = model(input_ph)
  model.add(Activation('softmax'))
  
  if logits:
    return model, logits_tensor, var_list
  else:
    return model, var_list

In [0]:
def mnist(train_start=0, train_end=60000, test_start=0,
                   test_end=10000, nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE,
                   learning_rate=LEARNING_RATE, train_dir=TRAIN_DIR,
                   filename=FILENAME, load_model=LOAD_MODEL,
                   testing=False, label_smoothing=0.1):
  """
  MNIST CleverHans tutorial
  :param train_start: index of first training set example
  :param train_end: index of last training set example
  :param test_start: index of first test set example
  :param test_end: index of last test set example
  :param nb_epochs: number of epochs to train model
  :param batch_size: size of training batches
  :param learning_rate: learning rate for training
  :param train_dir: Directory storing the saved model
  :param filename: Filename to save model under
  :param load_model: True for load, False for not load
  :param testing: if true, test error is calculated
  :param label_smoothing: float, amount of label smoothing for cross entropy
  :return: an AccuracyReport object
  """
  keras.layers.core.K.set_learning_phase(0)

  # Object used to keep track of (and return) key accuracies
  report = AccuracyReport()

  # Set TF random seed to improve reproducibility
  tf.set_random_seed(1234)

  if not hasattr(backend, "tf"):
    raise RuntimeError("This tutorial requires keras to be configured"
                       " to use the TensorFlow backend.")

  if keras.backend.image_dim_ordering() != 'tf':
    keras.backend.set_image_dim_ordering('tf')
    print("INFO: '~/.keras/keras.json' sets 'image_dim_ordering' to "
          "'th', temporarily setting to 'tf'")

  # Create TF session and set as Keras backend session
  sess = tf.Session()
  keras.backend.set_session(sess)

  # Get MNIST test data
  mnist = MNIST(train_start=train_start, train_end=train_end,
                test_start=test_start, test_end=test_end)
  x_train, y_train = mnist.get_set('train')
  x_test, y_test = mnist.get_set('test')
  
  x_train /= 255
  x_test /= 255

  # Obtain Image Parameters
  img_rows, img_cols, nchannels = x_train.shape[1:4]
  nb_classes = y_train.shape[1]

  # Define input TF placeholder
  x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,
                                        nchannels))
  y = tf.placeholder(tf.float32, shape=(None, nb_classes))
  
  x_train = x_train.reshape((-1,28*28))
  x_test = x_test.reshape((-1,28*28))
  x = tf.reshape(x,(-1,28*28))

  # Define TF model graph
  #model = cnn_model(img_rows=img_rows, img_cols=img_cols,
  #                  channels=nchannels, nb_filters=64,
  #                  nb_classes=nb_classes)
  """
  model = mlp_model(input_dim=img_rows*img_cols,
                    channels=nchannels, nb_units=64,
                    nb_classes=nb_classes)
  preds = model(x)
  
  print("Defined TensorFlow model graph.")

  def evaluate():
    # Evaluate the accuracy of the MNIST model on legitimate test examples
    eval_params = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds, x_test, y_test, args=eval_params)
    report.clean_train_clean_eval = acc
    print('Test accuracy on legitimate examples: %0.4f' % acc)

  # Train an MNIST model
  train_params = {
      'nb_epochs': nb_epochs,
      'batch_size': batch_size,
      'learning_rate': learning_rate,
      'train_dir': train_dir,
      'filename': filename
  }

  rng = np.random.RandomState([2017, 8, 30])
  if not os.path.exists(train_dir):
    os.mkdir(train_dir)

  ckpt = tf.train.get_checkpoint_state(train_dir)
  print(train_dir, ckpt)
  ckpt_path = False if ckpt is None else ckpt.model_checkpoint_path
  wrap = KerasModelWrapper(model)

  if load_model and ckpt_path:
    saver = tf.train.Saver()
    print(ckpt_path)
    saver.restore(sess, ckpt_path)
    print("Model loaded from: {}".format(ckpt_path))
    evaluate()
  else:
    print("Model was not loaded, training from scratch.")
    loss = CrossEntropy(wrap, smoothing=label_smoothing)
    train(sess, loss, x_train, y_train, evaluate=evaluate,
          args=train_params, rng=rng)

  # Calculate training error
  if testing:
    eval_params = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds, x_train, y_train, args=eval_params)
    report.train_clean_train_clean_eval = acc

  # Initialize the Fast Gradient Sign Method (FGSM) attack object and graph
  fgsm = FastGradientMethod(wrap, sess=sess)
  fgsm_params = {'eps': 0.3,
                 'clip_min': 0.,
                 'clip_max': 1.}
  adv_x = fgsm.generate(x, **fgsm_params)
  # Consider the attack to be constant
  adv_x = tf.stop_gradient(adv_x)
  preds_adv = model(adv_x)

  # Evaluate the accuracy of the MNIST model on adversarial examples
  eval_par = {'batch_size': batch_size}
  acc = model_eval(sess, x, y, preds_adv, x_test, y_test, args=eval_par)
  print('Test accuracy on adversarial examples: %0.4f\n' % acc)
  report.clean_train_adv_eval = acc

  # Calculating train error
  if testing:
    eval_par = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds_adv, x_train,
                     y_train, args=eval_par)
    report.train_clean_train_adv_eval = acc

  """
  # RP augmented mlp
  ######################################
  
  model, var_list = rp_mlp_model(input_dim=img_rows*img_cols,
                      channels=nchannels, nb_units=100,
                      nb_classes=nb_classes)

  preds = model(x)
  
  print("Defined TensorFlow model graph.")

  def evaluate():
    # Evaluate the accuracy of the MNIST model on legitimate test examples
    eval_params = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds, x_test, y_test, args=eval_params)
    report.clean_train_clean_eval = acc
    print('Test accuracy on legitimate examples: %0.4f' % acc)

  # Train an MNIST model
  train_params = {
      'nb_epochs': nb_epochs,
      'batch_size': batch_size,
      'learning_rate': learning_rate,
      'train_dir': train_dir,
      'filename': filename
  }

  rng = np.random.RandomState([2017, 8, 30])
  if not os.path.exists(train_dir):
    os.mkdir(train_dir)

  ckpt = tf.train.get_checkpoint_state(train_dir)
  print(train_dir, ckpt)
  ckpt_path = False if ckpt is None else ckpt.model_checkpoint_path
  wrap = KerasModelWrapper(model)

  if load_model and ckpt_path:
    saver = tf.train.Saver()
    print(ckpt_path)
    saver.restore(sess, ckpt_path)
    print("Model loaded from: {}".format(ckpt_path))
    evaluate()
  else:
    print("Model was not loaded, training from scratch.")
    loss = CrossEntropy(wrap, smoothing=label_smoothing)
    train(sess, loss, x_train, y_train, var_list=var_list, evaluate=evaluate,
          args=train_params, rng=rng)

  # Calculate training error
  if testing:
    eval_params = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds, x_train, y_train, args=eval_params)
    report.train_clean_train_clean_eval = acc

  # Initialize the Fast Gradient Sign Method (FGSM) attack object and graph
  fgsm = FastGradientMethod(wrap, sess=sess)
  fgsm_params = {'eps': 0.01,
                 'clip_min': 0.,
                 'clip_max': 1.}
  adv_x = fgsm.generate(x, **fgsm_params)
  # Consider the attack to be constant
  adv_x = tf.stop_gradient(adv_x)
  preds_adv = model(adv_x)

  # Evaluate the accuracy of the MNIST model on adversarial examples
  eval_par = {'batch_size': batch_size}
  acc = model_eval(sess, x, y, preds_adv, x_test, y_test, args=eval_par)
  print('Test accuracy on adversarial examples: %0.4f\n' % acc)
  report.clean_train_adv_eval = acc

  # Calculating train error
  if testing:
    eval_par = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds_adv, x_train,
                     y_train, args=eval_par)
    report.train_clean_train_adv_eval = acc
        
  global m 
  m = model
  return report

In [0]:
weights = None
def main(argv=None):
  #from cleverhans_tutorials import check_installation
  #check_installation(__file__)
  for i in range(0,5):
    report = mnist_tutorial(nb_epochs=NB_EPOCHS,
                       batch_size=BATCH_SIZE,
                       learning_rate=LEARNING_RATE,
                       train_dir=TRAIN_DIR,
                       filename=FILENAME,
                       load_model=LOAD_MODEL)

In [0]:
if __name__ == '__main__':
  """
  flags.DEFINE_integer('nb_epochs', NB_EPOCHS,
                       'Number of epochs to train model')
  flags.DEFINE_integer('batch_size', BATCH_SIZE, 'Size of training batches')
  flags.DEFINE_float('learning_rate', LEARNING_RATE,
                     'Learning rate for training')
  flags.DEFINE_string('train_dir', TRAIN_DIR,
                      'Directory where to save model.')
  flags.DEFINE_string('filename', FILENAME, 'Checkpoint filename.')
  flags.DEFINE_boolean('load_model', LOAD_MODEL,
                       'Load saved model or train.')
  """
  main()

Defined TensorFlow model graph.
train_dir None
Model was not loaded, training from scratch.
num_devices:  1


[INFO 2018-12-31 20:36:50,754 cleverhans] Epoch 0 took 1.1731157302856445 seconds


Test accuracy on legitimate examples: 0.1135


[INFO 2018-12-31 20:36:51,913 cleverhans] Epoch 1 took 0.9326636791229248 seconds


Test accuracy on legitimate examples: 0.3624


...

Test accuracy on legitimate examples: 0.7501


[INFO 2018-12-31 20:38:07,890 cleverhans] Epoch 8 took 0.9296507835388184 seconds


Test accuracy on legitimate examples: 0.7809


[INFO 2018-12-31 20:38:09,006 cleverhans] Epoch 9 took 0.9398238658905029 seconds


Test accuracy on legitimate examples: 0.7908
Test accuracy on adversarial examples: 0.0373



In [0]:
"""
Defined TensorFlow model graph.
train_dir None
Model was not loaded, training from scratch.
num_devices:  1
[INFO 2018-12-31 19:15:26,702 cleverhans] Epoch 0 took 1.125884771347046 seconds
Test accuracy on legitimate examples: 0.7984
[INFO 2018-12-31 19:15:27,795 cleverhans] Epoch 1 took 0.8888399600982666 seconds
Test accuracy on legitimate examples: 0.8848
[INFO 2018-12-31 19:15:28,855 cleverhans] Epoch 2 took 0.8946611881256104 seconds
Test accuracy on legitimate examples: 0.9120
[INFO 2018-12-31 19:15:29,910 cleverhans] Epoch 3 took 0.8911709785461426 seconds
Test accuracy on legitimate examples: 0.9177
[INFO 2018-12-31 19:15:30,964 cleverhans] Epoch 4 took 0.8903017044067383 seconds
Test accuracy on legitimate examples: 0.9256
[INFO 2018-12-31 19:15:32,022 cleverhans] Epoch 5 took 0.8943595886230469 seconds
Test accuracy on legitimate examples: 0.9277
[INFO 2018-12-31 19:15:33,074 cleverhans] Epoch 6 took 0.888146162033081 seconds
Test accuracy on legitimate examples: 0.9337
[INFO 2018-12-31 19:15:34,125 cleverhans] Epoch 7 took 0.8888247013092041 seconds
Test accuracy on legitimate examples: 0.9362
[INFO 2018-12-31 19:15:35,177 cleverhans] Epoch 8 took 0.8878788948059082 seconds
Test accuracy on legitimate examples: 0.9404
[INFO 2018-12-31 19:15:36,233 cleverhans] Epoch 9 took 0.8925678730010986 seconds
Test accuracy on legitimate examples: 0.9424
Test accuracy on adversarial examples: 0.0662
"""

'\nDefined TensorFlow model graph.\ntrain_dir None\nModel was not loaded, training from scratch.\nnum_devices:  1\n[INFO 2018-12-31 19:15:26,702 cleverhans] Epoch 0 took 1.125884771347046 seconds\nTest accuracy on legitimate examples: 0.7984\n[INFO 2018-12-31 19:15:27,795 cleverhans] Epoch 1 took 0.8888399600982666 seconds\nTest accuracy on legitimate examples: 0.8848\n[INFO 2018-12-31 19:15:28,855 cleverhans] Epoch 2 took 0.8946611881256104 seconds\nTest accuracy on legitimate examples: 0.9120\n[INFO 2018-12-31 19:15:29,910 cleverhans] Epoch 3 took 0.8911709785461426 seconds\nTest accuracy on legitimate examples: 0.9177\n[INFO 2018-12-31 19:15:30,964 cleverhans] Epoch 4 took 0.8903017044067383 seconds\nTest accuracy on legitimate examples: 0.9256\n[INFO 2018-12-31 19:15:32,022 cleverhans] Epoch 5 took 0.8943595886230469 seconds\nTest accuracy on legitimate examples: 0.9277\n[INFO 2018-12-31 19:15:33,074 cleverhans] Epoch 6 took 0.888146162033081 seconds\nTest accuracy on legitimate ex

In [0]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rp_layer1 (Dense)            (None, 80)                62720     
_________________________________________________________________
activation_21 (Activation)   (None, 80)                0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 80)                320       
_________________________________________________________________
dense_13 (Dense)             (None, 100)               8100      
_________________________________________________________________
activation_22 (Activation)   (None, 100)               0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 100)               400       
_________________________________________________________________
rp_layer2 (Dense)            (None, 80)                8000      
__________

In [0]:
# ensure rp layers not training
initial_weights = tf.Session().run(initial_weights)
final_weights = m.layers[0].get_weights()[0]
np.allclose(final_weights, initial_weights)

ValueError: ignored